This notebook shows how to quantize and run LLMs with mixed precision using ExLlamaV2.

More details in this article: [Run Llama 2 70B on Your GPU with ExLlamaV2](https://kaitchup.substack.com/p/run-llama-2-70b-on-your-gpu-with)

Note that when I wrote this notebook, ExLlamaV2 was still a very young project. If you find that it doesn't work anymore, please leave a comment in the article above. I'll update the notebook.

In [ ]:
!pip install transformers
!git clone https://github.com/turboderp/exllamav2
%cd exllamav2
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
Cloning into 'exllamav2'...
remote: Enumerating objects: 758, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 758 (delta 120), reused 135 (delta 98), pack-reused 569
Receiving objects: 100% (758/758), 1.22 MiB | 44.50 MiB/s, done.
Resolving deltas: 100% (470/470), done.
/content/exllamav2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.3 MB/s eta 0:00:00


We will download Llama 2 from the Hugging Face Hub. We must login.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

ExLlamaV2 doesn't communicate well with Hugging Face transformers. You first need to download the models locally. We only need the safetensors version. ExLlamaV2 uses safetensors so we don't need to download the ".bin" files.

In [ ]:
#The directory where we want to store the model must exist.
!mkdir ./Llama-2-13b-hf/

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id="meta-llama/Llama-2-13b-hf", ignore_patterns=["*.bin"], local_dir="./Llama-2-13b-hf/", local_dir_use_symlinks=False)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

'/content/exllamav2'

We need a dataset for calibrating the quantization. I use wikitext test set. Again, we need it locally so I download the .parquet file.

In [ ]:
!wget https://huggingface.co/datasets/wikitext/resolve/refs%2Fconvert%2Fparquet/wikitext-2-v1/test/0000.parquet

--2023-09-25 20:27:50--  https://huggingface.co/datasets/wikitext/resolve/refs%2Fconvert%2Fparquet/wikitext-2-v1/test/0000.parquet
Resolving huggingface.co (huggingface.co)... 13.35.166.69, 13.35.166.36, 13.35.166.50, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 685430 (669K)
Saving to: ‘0000.parquet’

0000.parquet        100%[===================>] 669.37K  1.69MB/s    in 0.4s    

2023-09-25 20:27:51 (1.69 MB/s) - ‘0000.parquet’ saved [685430/685430]



The quantization of Llama 2 13B is done with convert.py. It takes around 2 hours.

In [ ]:
!mkdir ./Llama-2-13b-hf/temp/
!python convert.py \
    -i ./ \
    -o ./Llama-2-13b-hf/temp/ \
    -c 0000.parquet \
    -cf ./Llama-2-13b-hf/3.0bpw/ \
    -b 3.0

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 -- 1.0:4b 32g s4                  4.13 bpw    rfn_error: 0.02717
 -- 0.1:5b/0.9:4b 32g s4           4.23 bpw    rfn_error: 0.02606
 -- 0.1:6b/0.9:4b 32g s4           4.33 bpw    rfn_error: 0.02580
 -- 1.0:5b 128g s4                 5.03 bpw    rfn_error: 0.01565
 -- 0.1:6b/0.9:5b 32g s4           5.23 bpw    rfn_error: 0.01348
 -- 0.05:8b/0.05:6b/0.9:5b 32g s4  5.33 bpw    rfn_error: 0.01338
 -- 0.4:6b/0.6:5b 32g s4           5.53 bpw    rfn_error: 0.01240
 -- 0.1:8b/0.3:6b/0.6:5b 32g s4    5.73 bpw    rfn_error: 0.01224
 -- 1.0:6b 128g s4                 6.03 bpw    rfn_error: 0.00823
 -- 1.0:6b 32g s4                  6.13 bpw    rfn_error: 0.00832
 -- 0.1:8b/0.9:6b 128g s4          6.23 bpw    rfn_error: 0.00786
 -- 1.0:8b 32g s4                  8.13 bpw    rfn_error: 0.00562
 -- Time: 8.82 seconds
 -- Linear: model.layers.16.self_attn.k_proj
 -- 0.05:3b/0.95:2b 32g s4         2.18 bpw    rfn_error: 0.09

You can use test_inference.py to try the model. In the remainder of this notebook, for the demonstration I use Llama 2 70B with an average 2.55 bpw. It was created by ExLlamaV2 authors.

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00


In [ ]:
!mkdir ./Llama-2-70b-2.5bpw/

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="turboderp/Llama2-70B-exl2", ignore_patterns=["*.bin"], revision="2.5bpw", local_dir="./Llama-2-70b-2.5bpw/", local_dir_use_symlinks=False)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

'/content/drive/MyDrive/Llama-2-70b-2.5bpw'

In [ ]:
!python test_inference.py -m ./Llama-2-70b-2.5bpw/ -p "Once upon a time,"

 -- Model: /content/drive/MyDrive//Llama-2-70b-2.5bpw/
 -- Options: ['rope_scale 1.0', 'rope_alpha 1.0']
 -- Loading model...
 -- Loading tokenizer...
 -- Warmup...
 -- Generating (greedy sampling)...

Once upon a time, there was a little girl who lived in a castle. She had everything a princess could want, and she was the most beautiful girl in the world.
One day, she looked in the mirror and said, “Mirror, mirror, on the wall, who is the fairest of them all?”
The mirror replied, “You are the most beautiful girl in the world.”
But the princess wasn’t satisfied. She wanted to be more beautiful. So she went to a witch and asked her to make her even more beautiful.
The witch gave her a potion, and the princess d

Prompt processed in 0.08 seconds, 5 tokens, 65.70 tokens/second
Response generated in 7.43 seconds, 128 tokens, 17.23 tokens/second
